<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_03_3_feature_encode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

**Module 3: Introduction to PyTorch**

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).


# Module 3 Material

- Part 3.1: Deep Learning and Neural Network Introduction [[Video]](https://www.youtube.com/watch?v=d-rU5IuFqLs&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_1_neural_net.ipynb)
- Part 3.2: Introduction to PyTorch [[Video]](https://www.youtube.com/watch?v=Pf-rrhMolm0&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_2_pytorch.ipynb)
- **Part 3.3: Encoding a Feature Vector for PyTorch Deep Learning** [[Video]](https://www.youtube.com/watch?v=7SGPm2tIT58&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_3_feature_encode.ipynb)
- Part 3.4: Early Stopping and Network Persistence [[Video]](https://www.youtube.com/watch?v=lS0vvIWiahU&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_4_early_stop.ipynb)
- Part 3.5: Sequences vs Classes in PyTorch [[Video]](https://www.youtube.com/watch?v=NOu8jMZx3LY&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_5_pytorch_class_sequence.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.


In [ ]:
try:
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Part 3.3: Encoding a Feature Vector for PyTorch Deep Learning

Neural networks can accept many types of data. We will begin with tabular data, where there are well-defined rows and columns. This data is what you would typically see in Microsoft Excel. Neural networks require numeric input. This numeric form is called a feature vector. Each input neurons receive one feature (or column) from this vector. Each row of training data typically becomes one vector. This section will see how to encode the following tabular data into a feature vector. You can see an example of tabular data below.


In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 7)
pd.set_option("display.max_rows", 5)

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=["NA", "?"],
)

pd.set_option("display.max_columns", 9)
pd.set_option("display.max_rows", 5)

display(df)

You can make the following observations from the above data:

- The target column is the column that you seek to predict. There are several candidates here. However, we will initially use the column "product". This field specifies what product someone bought.
- There is an ID column. You should exclude his column because it contains no information useful for prediction.
- Many of these fields are numeric and might not require further processing.
- The income column does have some missing values.
- There are categorical values: job, area, and product.

To begin with, we will convert the job code into dummy variables.


In [ ]:
pd.set_option("display.max_columns", 7)
pd.set_option("display.max_rows", 5)

dummies = pd.get_dummies(df["job"], prefix="job")
print(dummies.shape)

pd.set_option("display.max_columns", 9)
pd.set_option("display.max_rows", 10)

display(dummies)

Because there are 33 different job codes, there are 33 dummy variables. We also specified a prefix because the job codes (such as "ax") are not that meaningful by themselves. Something such as "job_ax" also tells us the origin of this field.

Next, we must merge these dummies back into the main data frame. We also drop the original "job" field, as the dummies now represent it.


In [ ]:
pd.set_option("display.max_columns", 7)
pd.set_option("display.max_rows", 5)

df = pd.concat([df, dummies], axis=1)
df.drop("job", axis=1, inplace=True)

pd.set_option("display.max_columns", 9)
pd.set_option("display.max_rows", 10)

display(df)

We also introduce dummy variables for the area column.


In [ ]:
pd.set_option("display.max_columns", 7)
pd.set_option("display.max_rows", 5)

df = pd.concat([df, pd.get_dummies(df["area"], prefix="area")], axis=1)
df.drop("area", axis=1, inplace=True)

pd.set_option("display.max_columns", 9)
pd.set_option("display.max_rows", 10)
display(df)

The last remaining transformation is to fill in missing income values.


In [ ]:
med = df["income"].median()
df["income"] = df["income"].fillna(med)

There are more advanced ways of filling in missing values, but they require more analysis. The idea would be to see if another field might hint at what the income was. For example, it might be beneficial to calculate a median income for each area or job category. This technique is something to keep in mind for the class Kaggle competition.

At this point, the Pandas dataframe is ready to be converted to Numpy for neural network training. We need to know a list of the columns that will make up _x_ (the predictors or inputs) and _y_ (the target).

The complete list of columns is:


In [ ]:
print(list(df.columns))

This data includes both the target and predictors. We need a list with the target removed. We also remove **id** because it is not useful for prediction.


In [ ]:
x_columns = df.columns.drop("product").drop("id")
print(list(x_columns))

## Generate X and Y for a Classification Neural Network

We can now generate _x_ and _y_. Note that this is how we generate y for a classification problem. Regression would not use dummies and would encode the numeric value of the target.


In [ ]:
from sklearn import preprocessing

# Convert to numpy - Classification
x_columns = df.columns.drop("product").drop("id")
x = df[x_columns].values
le = preprocessing.LabelEncoder()
y = le.fit_transform(df["product"])
products = le.classes_

We can display the _x_ and _y_ matrices.


In [ ]:
print(x)
print(y)

The x and y values are now ready for a neural network. Make sure that you construct the neural network for a classification problem. Specifically,

- Classification neural networks have an output neuron count equal to the number of classes.
- Classification neural networks should use **categorical_crossentropy** and a **softmax** activation function on the output layer.

## Generate X and Y for a Regression Neural Network

The program generates the _x_ values the say way for a regression neural network. However, _y_ does not use dummies. Make sure to replace **income** with your actual target.


In [ ]:
y = df["income"].values

# Module 3 Assignment

You can find the first assignment here: [assignment 3](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class3.ipynb)
